<a href="https://colab.research.google.com/github/roanacla/FactorsIntegration/blob/main/Factors_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone repos

In [1]:
#Content Statistics
!git clone https://github.com/Wayne122/Content_Statistics_Demo.git
%cd /content/Content_Statistics_Demo
!pip install -r requirements.txt
!python3 -m spacy download en_core_web_lg

#Sensationalism
%cd  /content/
!git clone https://github.com/roanacla/nlp_sensationalism_scorer.git
%cd /content/nlp_sensationalism_scorer/
!pip install -r requirements.txt

#Context Veracity
%cd  /content/
!git clone https://github.com/snarvekark/Veracity_Factor.git
%cd /content/Veracity_Factor/



Cloning into 'Content_Statistics_Demo'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 16 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.
/content/Content_Statistics_Demo
     |████████████████████████████████| 827.9MB 1.3MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=896699bc7bf75222452a42dace131874c28b83624e3365ba33fc4d3282b62a28
  Stored in directory: /tmp/pip-ephem-wheel-cache-psa0yi_1/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
/content
Cloning into 'nlp_sensationalism_scorer'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 10 (

restart runtime

# Download pretrained model

In [ ]:
%cd /content/
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

pretrained_model = drive.CreateFile({'id':"1oFjoL9LWrp2-YPSJL2UhBQ1efV9LiC2n"})
pretrained_model.GetContentFile('content_statistic_model.pickle')

/content


# Instantiation

In [ ]:
#Content Statistics
%cd /content/Content_Statistics_Demo/
from Blastoff_Content_Statistics import BlastoffContentStatistics
%cd /content/
bcs = BlastoffContentStatistics(mpath='content_statistic_model.pickle')

#Sensationalims
%cd /content/nlp_sensationalism_scorer/
from sensaScorer import SensaScorer
sensa = SensaScorer()

#Context Veracity
%cd /content/Veracity_Factor/
from context_veracity import Context_Veracity
cv = Context_Veracity()


# Run Polinomial Equation

In [ ]:
def isFake(text):
  accur = [0.26, 0.80, 0.50]
  w = [float(i)/sum(accur) for i in accur]
  
  prob = []

  if text:
    in_df = pd.DataFrame(data=[text], columns=['Statement'])
    res_cs = bcs.predict(in_df).replace('pants-fire', 1.0).replace('false', 0.8).replace('barely-true', 0.6).replace('half-true', 0.4).replace('mostly-true', 0.2).replace('true', 0.0)
    prob.append(w[0] * float(res_cs.iloc[0]))
    
  if text:
    prob.append(w[1] * cv.get_veracity_scores(text))
  if text:
    prob.append(w[2] * sensa.getScore(text))
    

  probTotal = sum(prob[0:len(prob)])
  return probTotal

In [ ]:
# result = isFake("Smartphones will become real personal computers")
# result = isFake("One America News Network, one of President Trump’s favorite media outlets, has been banned from posting new videos to YouTube for a week for spreading Covid-19 misinformation, YouTube said on Tuesday.")
result = isFake("Commander of the US Army: The elite of the US military will encircle China like the Soviet Union")
print("\n\n###### Result ######")
if result > 0.5:
  print("FAKE NEWS ALERT!!!")
else:
  print("NOT FAKE")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,




###### Result ######
FAKE NEWS ALERT!!!
